# 邏輯迴歸與決策邊界


<br>

## 目標

在本實驗中，你將會：

- 繪製邏輯迴歸模型的決策邊界，藉此更直觀地了解模型正在做出的預測。


In [ ]:
import sys, os
import numpy as np
try:
    %matplotlib widget
except:
    %matplotlib inline
    print("Colab not support matplotlib widget")
import matplotlib.pyplot as plt
from pathlib import Path

#region 匯入資料
def find_repo_root(marker="README.md"):
    cur = Path.cwd()
    while cur != cur.parent:  # 防止無限迴圈，到達檔案系統根目錄就停
        if (cur / marker).exists():
            return cur
        cur = cur.parent
    return None


def import_data_from_github():
    import os, urllib.request, pathlib, shutil
    
    def isRunningInColab() -> bool:
        return "google.colab" in sys.modules

    def isRunningInJupyterLab() -> bool:
        try:
            import jupyterlab
            return True
        except ImportError:
            return False
        
    def detect_env():
        from IPython import get_ipython
        if isRunningInColab():
            return "Colab"
        elif isRunningInJupyterLab():
            return "JupyterLab"
        elif "notebook" in str(type(get_ipython())).lower():
            return "Jupyter Notebook"
        else:
            return "Unknown"
        
    def get_utils_dir(env): 
        if env == "Colab": 
            if "/content" not in sys.path:
                sys.path.insert(0, "/content")
            return "/content/utils"
        else:
            return Path.cwd() / "utils"

    env = detect_env()
    UTILS_DIR = get_utils_dir(env)
    REPO_DIR = "Machine-Learning-Lab"

    #shutil.rmtree(UTILS_DIR, ignore_errors=True)
    os.makedirs(UTILS_DIR, exist_ok=True)

    BASE = f"https://raw.githubusercontent.com/mz038197/{REPO_DIR}/main"
    urllib.request.urlretrieve(f"{BASE}/utils/lab_utils_common_classification.py", f"{UTILS_DIR}/lab_utils_common_classification.py")
    urllib.request.urlretrieve(f"{BASE}/utils/plt_one_addpt_onclick.py", f"{UTILS_DIR}/plt_one_addpt_onclick.py")
    urllib.request.urlretrieve(f"{BASE}/utils/deeplearning.mplstyle", f"{UTILS_DIR}/deeplearning.mplstyle")


repo_root = find_repo_root()

if repo_root is None:
    import_data_from_github()
    repo_root = Path.cwd()
    

os.chdir(repo_root)
print(f"✅ 切換工作目錄至 {Path.cwd()}")
sys.path.append(str(repo_root)) if str(repo_root) not in sys.path else None
print(f"✅ 加入到系統路徑")

from utils.lab_utils_common_classification import plot_data, sigmoid, draw_vthresh

plt.style.use('utils/deeplearning.mplstyle')
print("✅ 匯入模組及設定繪圖樣式")
#endregion

<br>

## 資料集

假設你有以下的訓練資料集：
- 輸入變數 `X` 是一個 numpy 陣列，包含 6 筆訓練樣本，每筆樣本有兩個特徵
- 輸出變數 `y` 也是一個 numpy 陣列，包含 6 筆資料，且 `y` 的值為 `0` 或 `1`

In [ ]:
X = np.array([[0.5, 1.5], [1,1], [1.5, 0.5], [3, 0.5], [2, 2], [1, 2.5]])
y = np.array([0, 0, 0, 1, 1, 1]).reshape(-1,1) 

<br>

### 繪製資料

我們使用輔助函式來繪製這些資料點。標記為 $y=1$ 的資料點以紅色叉號顯示；標記為 $y=0$ 的資料點則以藍色圓點顯示。 

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(4,4))
plot_data(X, y, ax)

ax.axis([0, 4, 0, 3.5])
ax.set_ylabel('$x_1$')
ax.set_xlabel('$x_0$')
plt.show()

<br>

## 邏輯迴歸模型

* 假設你想用這些資料來訓練一個邏輯迴歸模型，其形式為：  

  $f(x) = g(w_0x_0+w_1x_1 + b)$
  
  其中 $g(z) = \frac{1}{1+e^{-z}}$，這個函數稱為 sigmoid（S 形）函數


* 假設你已經訓練好模型，並得到參數為 $b = -3, w_0 = 1, w_1 = 1$。也就是：

  $f(x) = g(x_0+x_1-3)$

  （你會在課程後續學到如何把這些參數擬合到資料上）
  
  
接著我們透過繪製它的決策邊界，來理解這個訓練後模型的預測結果。

<br>

### 回顧：邏輯迴歸與決策邊界

* 回想一下，對於邏輯迴歸，模型可表示為：

  $$f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = g(\mathbf{w} \cdot \mathbf{x}^{(i)} + b) \tag{1}$$

  其中 $g(z)$ 稱為 sigmoid 函數，它會把所有輸入值映射到 0 到 1 之間：

  $$ g(z) = \frac{1}{1+e^{-z}} \tag{2} $$
  
  而 $\mathbf{w} \cdot \mathbf{x}$ 是向量的內積（dot product）：
  
  $$\mathbf{w} \cdot \mathbf{x} = w_0 x_0 + w_1 x_1$$
  
  
* 我們將模型的輸出（$f_{\mathbf{w},b}(x)$）解讀為：在參數 $\mathbf{w}$ 與 $b$ 下、給定 $\mathbf{x}$ 時，$y=1$ 的機率。

* 因此，要從邏輯迴歸模型得到最終預測（$y=0$ 或 $y=1$），可以使用以下經驗法則：

  若 $f_{\mathbf{w},b}(x) >= 0.5$，則預測 $y=1$
  
  若 $f_{\mathbf{w},b}(x) < 0.5$，則預測 $y=0$
  
* 接著我們畫出 sigmoid 函數，看看在哪些區域 $g(z) >= 0.5$

In [ ]:
# Plot sigmoid(z) over a range of values from -10 to 10
z = np.arange(-10,11)

fig,ax = plt.subplots(1,1,figsize=(5,3))
# Plot z vs sigmoid(z)
ax.plot(z, sigmoid(z), c="b")

ax.set_title("Sigmoid function")
ax.set_ylabel('sigmoid(z)')
ax.set_xlabel('z')
draw_vthresh(ax,0)

* 如你所見，當 $z >=0$ 時，$g(z) >= 0.5$

* 對於邏輯迴歸模型，$z = \mathbf{w} \cdot \mathbf{x} + b$。因此：

  若 $\mathbf{w} \cdot \mathbf{x} + b >= 0$，模型會預測 $y=1$
  
  若 $\mathbf{w} \cdot \mathbf{x} + b < 0$，模型會預測 $y=0$


<br>

### 繪製決策邊界

現在，讓我們回到剛才的例子，來理解邏輯迴歸模型是如何做出預測的。

* 我們的邏輯迴歸模型形式為：

  $f(\mathbf{x}) = g(-3 + x_0+x_1)$


* 根據上面的結論，你可以看出：當 $-3 + x_0+x_1 >= 0$ 時，這個模型會預測 $y=1$

我們用圖形來看看它的樣子。我們先畫出 $-3 + x_0+x_1 = 0$，等價於 $x_1 = 3 - x_0$。

In [ ]:
# Choose values between 0 and 6
x0 = np.arange(0,6)

x1 = 3 - x0
fig,ax = plt.subplots(1,1,figsize=(5,4))
# Plot the decision boundary
ax.plot(x0,x1, c="b")
ax.axis([0, 4, 0, 3.5])

# Fill the region below the line
ax.fill_between(x0,x1, alpha=0.2)

# Plot the original data
plot_data(X,y,ax)
ax.set_ylabel(r'$x_1$')
ax.set_xlabel(r'$x_0$')
plt.show()

* 在上圖中，藍色線代表直線 $x_0 + x_1 - 3 = 0$。它應該會在 x1 軸的 3 處相交（令 $x_1$ = 3、$x_0$ = 0），也會在 x0 軸的 3 處相交（令 $x_1$ = 0、$x_0$ = 3）。


* 陰影區域表示 $-3 + x_0+x_1 < 0$；直線上方的區域則是 $-3 + x_0+x_1 > 0$。


* 陰影區域（直線下方）中的任何點都會被分類為 $y=0$；落在直線上或直線上方的點則會被分類為 $y=1$。這條線稱為「決策邊界」（decision boundary）。

如同你在課程中看到的，若加入更高次的多項式項（例如：$f(x) = g( x_0^2 + x_1 -1)$），就能得到更複雜的非線性邊界。

<br>

## 恭喜！
你已在邏輯斯迴歸的脈絡下探索了決策邊界。